In [2]:
import pandas as pd 
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#Cargar base de datos
df = pd.read_csv("C:\Users\ignac\Downloads\tareagit\nba_logreg2.csv", sep = ";")

In [ ]:
#Visualización 
print(df.head(5))
print(df.describe())

In [ ]:
#@title Selección de variables continuias
v_con = df.drop(columns = ['Name', 'TARGET_5Yrs'])
v_con
#@title Variables continuas estandarizadas y nuevo df de estas nuevas variables
#Instanciar y aplicar fit_transfor a dataset
data_std =  StandardScaler().fit_transform(v_con)

# Generar dataframe desde los datos estandarizados, agregando nombre de columnas.
df_data_std = pd.DataFrame(data_std, columns=v_con.columns)

df_data_std.iloc[:,:-1].apply(lambda x: np.mean(x).round(20))
df_data_std.iloc[:,:-1].apply(lambda x: np.var(x))

In [ ]:
#Aplicando el indice de Barlet el p-value es 0, por lo que a cualquier nivel de significancia se rechaza la hipotesis nula. Por lo tanto existe una relacion entre las variables.

In [ ]:
#@title Número de componentes criterio de Kaiser
autovalores = np.linalg.eigvals(np.corrcoef(df_data_std.T))

print(f'Los autovalores son los siguientes: {autovalores}')

# Filtro limite KMO
autovalores_kaiser = autovalores[autovalores>=0.8]

# Cantidad autovalores
cantidad_autovalores_kaiser = len(autovalores_kaiser)
print('\n')
print(f'La cantidad de componentes a utilizar asciende a {cantidad_autovalores_kaiser}')

In [ ]:
#@title Varianza explicada de los componentes
varianza_explicada=[]
varianza_explicada_acum =[]

# Loop para capturar la varianza explicada y la varianza explicada acumulada
for i in autovalores:
    varianza_explicada.append((i)/sum(autovalores)*100)
    varianza_explicada_acum.append(sum(varianza_explicada))


# Generar DataFrame
matriz_var_expl = pd.DataFrame({'pct_var_expl': varianza_explicada,
                                'pct_var_expl_acum': varianza_explicada_acum
                               },
                               index=np.linspace(1,len(autovalores), len(autovalores)))


# Usamos la cantidad de autovalores que nos arrojo el criterio de Kaiser.
matriz_var_expl.iloc[:,0:cantidad_autovalores_kaiser]

In [ ]:
#@title PCA según el criterio de Kaiser en un DataFrame con semilla 2021
# Criterio de Kaiser nos indica que son 4 las componentes.
pca = PCA(n_components=4)

In [ ]:
#@title Filt Transfor a dataset Normalizado.Generamos dataset reducido
pca_2 = pca.fit_transform(data_std)

In [ ]:
#@title Generar DataFrame de las componentes calculadas
columnas = ['CP1', 'CP2','CP3', 'CP4']
df_pca_2 = pd.DataFrame(pca_2 , columns=columnas)

# Data reducida en sus dimensionalidades
df_pca_2.head(5)

In [ ]:
# Generar DataFrame con los loading
df_loading = pd.DataFrame(pca.components_.T, columns=['CP 1', 'CP 2', 'CP 3', 'CP 4'], index=df_data_std.columns)
df_loading

In [ ]:
#@title Calculo de las componentes principales para cada valor
puntajes = np.dot(df_data_std, pca.components_.T)

In [ ]:
#@title Valor de las componentes principales para cada observación (principal component scores)
df_puntajes = pd.DataFrame(puntajes, columns=['CP 1', 'CP 2', 'CP 3', 'CP 4'] ,index = df_data_std.index)
df_puntajes